# IMPORTING MODULES

In [3]:
!pip install -r requirements.txt

  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.2
    Uninstalling protobuf-5.28.2:
      Successfully uninstalled protobuf-5.28.2
  Attempting uninstall: grpcio-status
    Found existing installation: grpcio-status 1.66.2
    Uninstalling grpcio-status-1.66.2:
      Successfully uninstalled grpcio-status-1.66.2


In [3]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn,config_context
patch_sklearn()
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
from scipy.stats import chi2_contingency
import random
import google.generativeai as genai
import os
from dotenv import load_dotenv
import json
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import re
import numpy as np
from sklearn.svm import SVC
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
import sys
import warnings
warnings.filterwarnings("ignore", message="`crosstab` is not currently supported by PandasOnRay, defaulting to pandas implementation.")


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# IMPORTING FILES & PREPROCESSING

In [4]:
df = pd.read_csv('Training.csv')
column_names1 = ['disease', 'reccomendation1', 'reccomendation2','reccomendation3','reccomendation4']
column_names2 = ['disease', 'description']
df_prec = pd.read_csv('symptom_precaution.csv',names=column_names1,header=0)
df_desc = pd.read_csv('symptom_Description.csv',names=column_names2,header=0)

In [5]:
columns_list = df.columns.tolist()
df1=df.copy()
columns_list[0]=columns_list[-1]
columns_list.pop(-1)
df = df[columns_list]
df1 = df.copy()
df1=df1[columns_list]

In [6]:
# Encode the target 'prognosis'
label_encoder = LabelEncoder()
df['prognosis'] = label_encoder.fit_transform(df['prognosis'])

# Split data into features (X) and target (y)
X = df.drop('prognosis', axis=1)  # all symptoms as features
y = df['prognosis']  # prognosis as the target

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Train a Random Forest Classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)  # get probabilities


In [7]:
user_symptoms=[]
symptoms = ['skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish_skin', 'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever', 'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes', 'malaise', 'blurred_and_distorted_vision', 'phlegm', 'throat_irritation', 'redness_of_eyes', 'sinus_pressure', 'runny_nose', 'congestion', 'chest_pain', 'weakness_in_limbs', 'fast_heart_rate', 'pain_during_bowel_movements', 'pain_in_anal_region', 'bloody_stool', 'irritation_in_anus', 'neck_pain', 'dizziness', 'cramps', 'bruising', 'obesity', 'swollen_legs', 'swollen_blood_vessels', 'puffy_face_and_eyes', 'enlarged_thyroid', 'brittle_nails', 'swollen_extremeties', 'excessive_hunger', 'extra_marital_contacts', 'drying_and_tingling_lips', 'slurred_speech', 'knee_pain', 'hip_joint_pain', 'muscle_weakness', 'stiff_neck', 'swelling_joints', 'movement_stiffness', 'spinning_movements', 'loss_of_balance', 'unsteadiness', 'weakness_of_one_body_side', 'loss_of_smell', 'bladder_discomfort', 'foul_smell_of urine', 'continuous_feel_of_urine', 'passage_of_gases', 'internal_itching', 'toxic_look_(typhos)', 'depression', 'irritability', 'muscle_pain', 'altered_sensorium', 'red_spots_over_body', 'belly_pain', 'abnormal_menstruation', 'dischromic _patches', 'watering_from_eyes', 'increased_appetite', 'polyuria', 'family_history', 'mucoid_sputum', 'rusty_sputum', 'lack_of_concentration', 'visual_disturbances', 'receiving_blood_transfusion', 'receiving_unsterile_injections', 'coma', 'stomach_bleeding', 'distention_of_abdomen', 'history_of_alcohol_consumption', 'fluid_overload.1', 'blood_in_sputum', 'prominent_veins_on_calf', 'palpitations', 'painful_walking', 'pus_filled_pimples', 'blackheads', 'scurring', 'skin_peeling', 'silver_like_dusting', 'small_dents_in_nails', 'inflammatory_nails', 'blister', 'red_sore_around_nose', 'yellow_crust_ooze']
filtered_df=""
def create_symptom_vector(user_symptoms):
        symptom_vector = [0] * len(symptoms)
    
        user_symptoms = [symptom.strip().lower() for symptom in user_symptoms]
    
        for symptom in user_symptoms:
            if symptom in symptoms:
                idx = symptoms.index(symptom)
                symptom_vector[idx] = 1
            else:
                print(f"Symptom '{symptom}' not recognized.")
        
        return symptom_vector
    
def get_top_diseases(proba, n=40):
        # Get top N diseases with highest probabilities
        top_diseases_idx = proba.argsort()[::-1][:n]
        top_diseases = label_encoder.inverse_transform(top_diseases_idx)
        top_probs = proba[top_diseases_idx]
        return list(zip(top_diseases, top_probs))

def predict_dis(user_symptoms):
    new_symptoms = create_symptom_vector(user_symptoms)
    new_symptoms_df = pd.DataFrame([new_symptoms], columns=symptoms)
    proba = clf.predict_proba(new_symptoms_df)
    dynamic_prob=0.0
    top_diseases_with_probs = get_top_diseases(proba[0])
    top_diseases=[]
    for disease, prob in top_diseases_with_probs:
        if prob>dynamic_prob:
            #print(f"Disease: {disease}, Probability: {prob:.4f}")
            top_diseases.append(disease)
    
    filtered_df = df1[df1['prognosis'].isin(list(top_diseases))]
    return filtered_df
    
def predict_dis_2(user_symptoms):
    new_symptoms = create_symptom_vector(user_symptoms)
    new_symptoms_df = pd.DataFrame([new_symptoms], columns=symptoms)
    proba = clf.predict_proba(new_symptoms_df)
    
    top_diseases_with_probs = get_top_diseases(proba[0])
    top_diseases=[]
    dynamic_prob=0.1
    for disease, prob in top_diseases_with_probs:
        if prob>dynamic_prob:
            #print(f"Disease: {disease}, Probability: {prob:.4f}")
            top_diseases.append(disease)
    
    filtered_df = df1[df1['prognosis'].isin(list(top_diseases))]
    return top_diseases





In [8]:

def rank_symptoms(dfa):
    symptom_ranking = {}
    
    # Perform Chi-Square test for each symptom
    for symptom in dfa.columns:
        contingency_table = pd.crosstab(dfa.index, dfa[symptom])
        
        # Ensure the contingency table has data and at least two unique values
        if len(contingency_table) > 1 and contingency_table.values.sum() > 0:
            chi2, p, dof, expected = chi2_contingency(contingency_table)
            symptom_ranking[symptom] = chi2  # Use Chi-Square value as importance score
        else:
            # If contingency table is empty or invalid, skip the symptom
            print(f"Skipping symptom '{symptom}' due to insufficient data.")

    # Sort symptoms by importance (highest Chi-Square first)
    ranked_symptoms = sorted(symptom_ranking.items(), key=lambda x: x[1], reverse=True)
    return ranked_symptoms


def rerank(filtered_df):
    ranked_symptoms=[]
    rs = rank_symptoms(filtered_df)[1:]
    for i in range(len(rs)):
        if rs[i][-1]!=0:
            ranked_symptoms.append(rs[i])
    return ranked_symptoms

# PREPARING LLMS AND HELPER FUNCTIONS (TO GET DISEASE INPUT)

In [9]:

def create_gemini_response(disease_name): #GEMINI

    load_dotenv('apiee.env')
    
    my_key = os.getenv("API_KEY")
    genai.configure(api_key=my_key)
    model = genai.GenerativeModel('gemini-pro')
    disease_name=disease_name[0]
    rows_desc = df_desc[df_desc['disease'] == disease_name]
    rows_prec = df_prec[df_prec['disease'] == disease_name]
    # Extract the description as a single string
    description = rows_desc['description'].values[0] if not rows_desc.empty else "No description found."
    
    # Extract all precautions as a single string
    if not rows_prec.empty:
        precautions = ' '.join(rows_prec.iloc[0, 1:].values)  # Joining all recommendations from the first row
    else:
        precautions = "No precautions found."
    
    
    disease_description = df_desc
    df_prec
    response = model.generate_content(f"""
    You are an AI assistant specializing in providing detailed health information. Given the following details about a disease, generate a comprehensive summary.

    1. **Disease Name**: {disease_name}

    2. **Description**: {description}

    3. **Health Precautions**: {precautions}

    Please ensure that the summary is:

    - Easy to understand for a general audience.
    - Detailed enough to cover the nature of the disease and its associated health precautions.
    
    **Important**: Include the following disclaimer at the end of your summary:

    Disclaimer: This summary is generated based on limited data available in the dataset. The information provided here is for informational purposes only and should not be considered medical advice. Always consult a healthcare professional for personalized diagnosis and treatment options.
    """)
    return response.text


def gemini_answer_user_question(disease_name,user_question):
    FOLLOWUP_QUESTION_PROMPT_TEMPLATE = """
    The user has follow-up questions regarding the predicted disease.
    
    Predicted disease: {disease_name}
    
    User's question: {user_question}
    
    ---
    
    Provide a detailed and accurate answer to the user's question based on the predicted disease. Make sure to give a clear response and, if applicable, include sources or disclaimers where necessary.
    
    Answer:
    """
    prompt = FOLLOWUP_QUESTION_PROMPT_TEMPLATE.format(
        disease_name=disease_name,
        user_question=user_question
    )

    # Initialize the model
    load_dotenv('apiee.env')
    
    my_key = os.getenv("API_KEY")
    genai.configure(api_key=my_key)
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    

    return response.text

In [10]:

def create_medllama_response(disease_name): #GEMINI

    model = Ollama(model="llama3.2_mod")
    disease_name=disease_name[0]
    rows_desc = df_desc[df_desc['disease'] == disease_name]
    rows_prec = df_prec[df_prec['disease'] == disease_name]
    # Extract the description as a single string
    description = rows_desc['description'].values[0] if not rows_desc.empty else "No description found."
    
    # Extract all precautions as a single string
    if not rows_prec.empty:
        precautions = ' '.join(rows_prec.iloc[0, 1:].values)  # Joining all recommendations from the first row
    else:
        precautions = "No precautions found."
    
    
    disease_description = df_desc
    response = model.invoke(f"""
    You are an AI assistant specializing in providing detailed health information. Given the following details about a disease, generate a comprehensive summary.

    1. **Disease Name**: {disease_name}

    2. **Description**: {description}

    3. **Health Precautions**: {precautions}

    Please ensure that the summary is:

    - Easy to understand for a general audience.
    - Detailed enough to cover the nature of the disease and its associated health precautions.
    
    **Important**: Include the following disclaimer at the end of your summary:

    Disclaimer: This summary is generated based on limited data available in the dataset. The information provided here is for informational purposes only and should not be considered medical advice. Always consult a healthcare professional for personalized diagnosis and treatment options.
    """)
    return response




PROMPT_TEMPLATE = """
Based on the user's input, predict the most likely disease from the provided 'Possible diseases after initial analysis' and provide necessary disclaimers.
If there is a more precise prediction from the symptoms, indicate that also.
User-reported symptoms: {user_symptoms} - These symptoms are present

Symptoms not present: {symptoms_not_present} - These symptoms are not present.

Possible diseases: {possible_diseases}

---

Make a prediction about the most likely disease the user might have, considering the context provided. 
Explain why and how you came to a conclusion, and how confident you are.
Additionally, provide a disclaimer that this is a prediction and should not be considered a definitive medical diagnosis.

Include sources if relevant.

Answer:
"""

def run_medllama_multiple_diseases(user_symptoms,symptoms_not_present,possible_diseases):
    # Example inputs from the user for testing

    # Construct the full prompt by filling in the template
    prompt = PROMPT_TEMPLATE.format(
        user_symptoms=user_symptoms,
        symptoms_not_present=symptoms_not_present,
        possible_diseases=possible_diseases
    )

    # Initialize the model
    model = Ollama(model="medllama2_mod")
    
    # Generate the response from the model
    response_text = model.invoke(prompt)

    return response_text
    

# follow up questions
FOLLOWUP_QUESTION_PROMPT_TEMPLATE = """
The user has follow-up questions regarding the predicted disease.

Predicted disease: {disease_name}


User's question: {user_question}

---

Provide a detailed and accurate answer to the user's question based on the predicted disease. Make sure to give a clear response and, if applicable, include sources or disclaimers where necessary.

Answer:
"""

def medllama_answer_user_question(disease_name, user_question):
    # Construct the prompt for follow-up questions
    prompt = FOLLOWUP_QUESTION_PROMPT_TEMPLATE.format(
        
        disease_name=disease_name,
        user_question=user_question
    )

    # Initialize the model
    model = Ollama(model="medllama2_mod")
    

    
    # Generate the response from the model with specified parameters
    response_text = model.invoke(prompt)
    

    return response_text
    



In [11]:
with open('combined_intents.json', 'r') as file:
    data = json.load(file)

# Step 2: Prepare the dataset
X = []
y = []

# Extract patterns and corresponding tags from the JSON
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        #print(pattern+' '+intent["tag"])
        X.append(pattern+' '+intent["tag"])
        y.append(intent["tag"])

# Step 3: Preprocessing function to normalize user input and patterns
def preprocess_text(text):
    text = text.strip().lower()  # Convert to lowercase and remove extra spaces
    return re.sub(r'[^\w\s]', '', text)  # Remove punctuation

# Apply preprocessing to the patterns
X = [preprocess_text(pattern) for pattern in X]

# Step 4: Convert text patterns into numerical vectors using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2), stop_words='english')
X_vec = vectorizer.fit_transform(X).toarray()

# Step 5: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.3, random_state=42)

# Step 6: Train the SVM model
svm_clf = SVC(kernel='linear', probability=True)  # Linear kernel for text classification
svm_clf.fit(X_train, y_train)

# Step 7: Make predictions and evaluate the model
y_pred = svm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy on test data: {accuracy * 100:.2f}%")



Model accuracy on test data: 98.62%


In [12]:
def predict_tag(user_symptom):
    # Step 8a: Preprocess the user symptom
    user_symptom = preprocess_text(user_symptom)
    
    # Step 8b: Convert user input into a TF-IDF vector
    user_symptom_vec = vectorizer.transform([user_symptom]).toarray()
    
    # Step 8c: Predict the tag using the trained SVM model
    predicted_tag = svm_clf.predict(user_symptom_vec)
    probabilities = svm_clf.predict_proba(user_symptom_vec)
    
    # Output the predicted tag and probabilities
    return predicted_tag[0]

# QUESTIONAIRE MODULE

In [ ]:
correct,wrong,semi,average= 0,0,0,0
avg_sy = 0
avg_qn = 0
possible_diseases = []

#print(symptoms)

def predict_and_rerank(user_symptoms):
    """Predict possible diseases and rerank symptoms based on current symptoms."""
    filtered_df = predict_dis(user_symptoms)
    ranked_symptoms = rerank(filtered_df) if not filtered_df.empty else []
    possible_diseases = predict_dis_2(user_symptoms)
    return ranked_symptoms, possible_diseases
possible_diseases=[]
absent_symp=[]
def ask_questionnaire(df2, ranked_symptoms, relevant_symptoms):
    global user_symptoms, avg_qn, count,possible_diseases,absent_symp
    
    count = 0
    no = 0
    max_no_of_questions=25 #change number of qns
    while no < len(ranked_symptoms):
        symptom = ranked_symptoms[no][0]
        no += 1
        if count>=max_no_of_questions:
            return
        if symptom not in user_symptoms and symptom not in absent_symp:
            a = input(f"Do you have {symptom}? (yes/no): ").strip().lower()
            answer = 'yes' if a == 'yes' else 'no'
            count += 1
            if len(possible_diseases) == 1:
                return
            if answer == 'yes':
                user_symptoms.append(symptom)
                ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)
                no = 0  # Reset if new symptoms are added
                #print(possible_diseases)
            if answer == 'no':
                absent_symp.append(symptom)
                # Remove diseases where the symptom has a 'yes' (1) value
                temp=possible_diseases.copy()
                possible_diseases = [
                    disease for disease in possible_diseases 
                    if df2[df2['prognosis'] == disease][symptom].values[0] == 0
                ]
                if possible_diseases != temp:
                    ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)
                    #print(possible_diseases)
                    no = 0
                
            if len(possible_diseases) == 1:
                return
            
    # If still multiple diseases remain
    avg_qn += count

def do():
    global correct,wrong,semi,average
    global avg_sy, avg_qn, user_symptoms
    yn='n'
    while yn!='y':
        user_symptoms = []
        for c in range(4):
            print("\nEnter your symptoms (one at a time):")
            user_input = input().strip()
            user_symptoms.append(predict_tag(user_input))
        
        print(user_symptoms, ": Are the symptoms you entered.")
        yn = input("Is this correct? (y/n): ").strip().lower()
    
        if yn == 'y':
            break  # Exit the loop if the input is correct
        elif yn == 'n':
            print("Let's modify your symptoms.")
            modify='y'
            
            while True:
            
                if modify == 'y':
                    index = int(input("Enter the index of the symptom to modify (1 to {}): ".format(len(user_symptoms))))
                    if 0 < index <= len(user_symptoms):
                        new_symptom = input("Enter the new symptom: ").strip()
                        user_symptoms[index-1] = new_symptom
                        print(user_symptoms, ": Are the symptoms you entered.")
                        yn = input("Is this correct? (y/n): ").strip().lower()
                    
                        if yn == 'y':
                            break  # Exit the loop if the input is correct
                    else:
                        print("Invalid index. Please try again.")
                else:
                    print("Final symptoms confirmed:", user_symptoms)
                    break  # Exit the modification loop
        else:
            print("Invalid input. Please enter 'y' or 'n'.")
    
    # Proceed with the confirmed user symptoms
    print("Proceeding with the following symptoms:", user_symptoms)

        
    # Predict possible diseases based on initial symptoms
    ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)
    print(ranked_symptoms)
    if len(possible_diseases) == 1:
        predicted_disease = possible_diseases[0]
        # Print only the necessary details
        print(f"Predicted disease: {predicted_disease} (no questions asked)")
        return [predicted_disease]
    
    # If multiple diseases remain, start asking questions
    if len(possible_diseases) > 1:
        ask_questionnaire(df1, ranked_symptoms, possible_diseases)
        
        # Check the number of possible diseases after questioning
        ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)  # Re-evaluate possible diseases after questioning
        
        if len(possible_diseases) == 1:  # If only one disease remains after questioning
            predicted_disease = possible_diseases[0]
            print(f"Predicted disease: {predicted_disease}, identified in {count} questions")
            return [predicted_disease]
        else:  # Still multiple diseases remain
            print(f"Possible diseases after questioning: {possible_diseases}")
            response_text=run_medllama_multiple_diseases(user_symptoms,absent_symp,str(possible_diseases))
            print(f"Model response:\n{response_text}")
            return possible_diseases

    else:
        print("Unknown disease", count)
        sys.exit()
        
disease_name=do()
    
ch=input("Do you want more information? (y/n): ")

if ch.lower()=='y':
    
        m_name=input("Do you want to use Gemini or MedLLaMA Model? :")

        if len(disease_name)==1:
            u_disease=disease_name[0]
        else:
            u_disease=input("Enter the disease name to get more information on the same: ")

        if m_name.lower()=='gemini':
            description=create_gemini_response([u_disease])
            print("Model response: ",description)
            
            qn=input("Feel free to ask me any following questions here: ")
            while qn.lower()!='no questions':
                print(gemini_answer_user_question(u_disease,qn))
                qn=input("Feel free to ask me any following questions here('no questions' to stop):\n ")

    
            
        if m_name.lower()=='medllama':
            description=create_medllama_response([u_disease])
            print("Model response: ",description)
            
            qn=input("Feel free to ask me any following questions here: ")
            while qn.lower()!='no questions' or qn.lower()!='no question' or qn.lower()!='no':
                print(medllama_answer_user_question(u_disease, qn))
                qn=input("Feel free to ask me any following questions here('no questions' to stop):\n ")
                if qn.lower() in ['no questions','no','no question']:
                    break
                
    


Enter your symptoms (one at a time):


 fatigue



Enter your symptoms (one at a time):


 throat pain



Enter your symptoms (one at a time):


...


Enter your symptoms (one at a time):


 malaise


['fatigue', 'throat_irritation', 'high_fever', 'malaise'] : Are the symptoms you entered.


Is this correct? (y/n):  y


Proceeding with the following symptoms: ['fatigue', 'throat_irritation', 'high_fever', 'malaise']
[('joint_pain', 2880.000000000001), ('muscle_wasting', 2880.000000000001), ('mood_swings', 2880.000000000001), ('patches_in_throat', 2880.000000000001), ('dark_urine', 2880.000000000001), ('constipation', 2880.000000000001), ('diarrhoea', 2880.000000000001), ('malaise', 2880.000000000001), ('blurred_and_distorted_vision', 2880.000000000001), ('throat_irritation', 2880.000000000001), ('redness_of_eyes', 2880.000000000001), ('sinus_pressure', 2880.000000000001), ('runny_nose', 2880.000000000001), ('congestion', 2880.000000000001), ('bloody_stool', 2880.000000000001), ('irritation_in_anus', 2880.000000000001), ('neck_pain', 2880.000000000001), ('obesity', 2880.000000000001), ('enlarged_thyroid', 2880.000000000001), ('brittle_nails', 2880.000000000001), ('swollen_extremeties', 2880.000000000001), ('extra_marital_contacts', 2880.000000000001), ('swelling_joints', 2880.000000000001), ('unsteadin

# RANDOM DISEASE GENERATOR FOR TESTING

In [22]:
random_index = np.random.randint(0, len(df1))
selected_disease = df1.iloc[random_index]
print(selected_disease['prognosis'])
# Determine relevant symptoms for the selected disease
relevant_symptoms = [col for col in df1.columns if selected_disease[col] == 1]
print(sorted(relevant_symptoms))

Typhoid
['belly_pain', 'chills', 'constipation', 'diarrhoea', 'fatigue', 'headache', 'high_fever', 'nausea', 'toxic_look_(typhos)', 'vomiting']


# AUTOMATED PREDICTION (TESTING PURPOSES ONLY)

In [12]:

correct,wrong,semi,average= 0,0,0,0
avg_sy = 0
avg_qn = 0
print("Randomly generated disease,Predicted Disease(s),Questions asked")
# Helper functions
def get_random_symptom(relevant_symptoms, current_symptoms):
    """Select a random symptom not already in current symptoms."""
    new_symptoms = [symptom for symptom in relevant_symptoms if symptom not in current_symptoms]
    if new_symptoms:  # Check if there are any new symptoms to choose from
        return random.choice(new_symptoms)
    return None

def predict_and_rerank(user_symptoms):
    """Predict possible diseases and rerank symptoms based on current symptoms."""
    filtered_df = predict_dis(user_symptoms)
    ranked_symptoms = rerank(filtered_df) if not filtered_df.empty else []
    possible_diseases = predict_dis_2(user_symptoms)
    return ranked_symptoms, possible_diseases

def ask_questionnaire(ranked_symptoms, relevant_symptoms, possible_diseases):
    global df1,user_symptoms, avg_qn, count
    absent_symp=[]
    count = 0
    no = 0
    max_no_of_questions=25 #change number of qns
    while no < len(ranked_symptoms):
        symptom = ranked_symptoms[no][0]
        no += 1
        if count>=max_no_of_questions:
            return
        if symptom not in user_symptoms and symptom not in absent_symp:
            answer = 'yes' if symptom in relevant_symptoms else 'no'
            count += 1
            # If we've narrowed it down to one disease, stop asking
            

            # Update symptoms and predict again if 'yes'
            if answer == 'yes':
                user_symptoms.append(symptom)
                ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)
                no = 0  # Reset if new symptoms are added
            if answer == 'no':
                absent_symp.append(symptom)
                # Remove diseases where the symptom has a 'yes' (1) value
                temp=possible_diseases.copy()
                possible_diseases = [
                    disease for disease in possible_diseases 
                    if df1[df1['prognosis'] == disease][symptom].values[0] == 0
                ]
                if possible_diseases != temp:
                    ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)
                    #print(possible_diseases)
                    no = 0
            if len(possible_diseases) == 1:
                return

    # If still multiple diseases remain
    avg_qn += count

def do(number_of_questions):
    global correct,wrong,semi,average
    global avg_sy, avg_qn, user_symptoms

    # Randomly select a disease
    random_index = np.random.randint(0, len(df1))
    selected_disease = df1.iloc[random_index]

    # Determine relevant symptoms for the selected disease
    relevant_symptoms = [col for col in df1.columns if selected_disease[col] == 1]

    user_symptoms = []  # Ensure this is initialized as a list
    # Start with three random symptoms
    for _ in range(number_of_questions):
        initial_symptom = get_random_symptom(relevant_symptoms, user_symptoms)
        if initial_symptom:
            user_symptoms.append(initial_symptom)

    # Predict possible diseases based on initial symptoms
    ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)

    if len(possible_diseases) == 1:
        predicted_disease = possible_diseases[0]
        # Print only the necessary details
        print(selected_disease['prognosis'], predicted_disease, 0)
        if str(selected_disease['prognosis'])==str(predicted_disease):
                correct+=1
        else:
                wrong+=1
        return
    
    # If multiple diseases remain, start asking questions
    if len(possible_diseases) > 1:
        ask_questionnaire(ranked_symptoms, relevant_symptoms, possible_diseases)
        
        # Check the number of possible diseases after questioning
        ranked_symptoms, possible_diseases = predict_and_rerank(user_symptoms)  # Re-evaluate possible diseases after questioning

        if len(possible_diseases) == 1:  # If only one disease remains after questioning
            predicted_disease = possible_diseases[0]
            print(selected_disease['prognosis'], predicted_disease, count)
            if str(selected_disease['prognosis'])==str(predicted_disease):
                correct+=1
            else:
                wrong+=1
        else:  # Still multiple diseases remain
            print(selected_disease['prognosis'], possible_diseases, count)
            if selected_disease['prognosis'] in possible_diseases:
                semi+=1
            else:
                wrong+=1

    else:
        # If the count didn't change during questioning
        print(selected_disease['prognosis'], "Unknown", count)
        wrong+=1
    average+=count
    
# Example invocation
for i in range(20):
    do(3) #number of symptoms entered
correctperc=correct/(correct+wrong+semi)
wrongperc=wrong/(correct+wrong+semi)
semiperc=semi/(correct+wrong+semi)
avgqns=average/20
print(correctperc,"-Correct percentage|",wrongperc,"-Wrong percentage|",semiperc,"-Semi correct percentage|",avgqns,"-Average no of questions")

Randomly generated disease,Predicted Disease(s),Questions asked
Tuberculosis ['GERD', 'Chronic cholestasis', 'Heart attack'] 25
Psoriasis Psoriasis 0
Peptic ulcer diseae Peptic ulcer diseae 14
Typhoid ['Typhoid', 'Dimorphic hemmorhoids(piles)'] 25
Hyperthyroidism ['Hyperthyroidism', 'Jaundice', 'Arthritis'] 25
Cervical spondylosis Cervical spondylosis 0
Common Cold ['Common Cold', 'GERD'] 25
Peptic ulcer diseae ['Peptic ulcer diseae', 'Chronic cholestasis'] 25
Peptic ulcer diseae Peptic ulcer diseae 0
Gastroenteritis Gastroenteritis 0
Hyperthyroidism Hyperthyroidism 11
Common Cold ['Common Cold', 'Allergy'] 25
Malaria Malaria 0
Arthritis Arthritis 0
Migraine Migraine 23
Hypoglycemia Hypoglycemia 24
Pneumonia Chicken pox 0
Allergy Allergy 0
Fungal infection Fungal infection 0
Chicken pox Chicken pox 14
0.65 -Correct percentage| 0.1 -Wrong percentage| 0.25 -Semi correct percentage| 11.8 -Average no of questions
